# PROMPT RESULT ANALYSIS

In [ ]:
import pickle
import os
import json
from eval import choose_plan

pred_plans = pickle.load(open('result/alfred/prompt10/vanilla/valid_seen_2.p'))

not_executable = []
correct = []

for goal in pred_plans:
    best_plan = choose_plan(pred_plans[goal]['plan'])
    for ep in os.listdir('data/alfred_data/json_2.1.0/valid_seen'):
        for trial in os.listdir(os.path.join('data/alfred_data/json_2.1.0/valid_seen', ep)):
            traj_data = json.load(open(os.path.join('data/alfred_data/json_2.1.0/valid_seen', ep, trial, 'traj_data.json')))
            anns = [ann['task_desc'] for ann in traj_data['turk_annotations']['anns']]
            if goal not in anns:
                continue
            task_type = traj_data['task_type']
            pddl_param = traj_data['pddl_params']
            if best_plan is None:
                not_executable.append({
                    'goal': goal,
                    'task_type': task_type,
                    'pddl_param':pddl_param
                    })
            


# HuggingFace 써보기

## pipeline

자동으로 기본 모델과 preprocessing을 사용해서 inference한다.

In [ ]:
from transformers import pipeline

generator = pipeline() 
# task='', model='',  -- huggingface hub에 있는대로 가져와야 함. ex) openai/whisper-large
# task specific parameters는 task API 확인. ex) transformers.[taskname]Pipeline.call()의 파라미터
result = generator(input_data) # 여러개면 list of data

In [2]:
from transformers import pipeline

generator = pipeline(model='gpt2', device=0)
print(generator('New way to solve the problem is'))

/home/user/anaconda3/envs/language-planner/lib/python3.7/site-packages/transformers/generation/utils.py:1212: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  "You have modified the pretrained model configuration to control generation. This is a"
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/user/anaconda3/envs/language-planner/lib/python3.7/site-packages/transformers/generation/utils.py:1302: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


[{'generated_text': 'New way to solve the problem is to give your children a new school that will be similar to yours. The first step is to set up a new school that will teach English and you must send him to a similar school in a different country. The'}]


## 지정해서 사용하기
pipeline = tokenizer + model

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained() # 모델 이름
encoded_input = tokenizer('') # input, padding, truncation, return_tensors='pt', ...

decoding method
* greedy search (default)
* contrastive search - non-repetitive, long
* beam-search decoding - 전체 sequence를 참조해서 가장 prob 높은 것을 고른다, num beams는 keep하는 hypothesis의 수

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = 'gpt2-xl' # gpt2-xl
tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer('Correct this to standard English:\n slice WineBottle\n', return_tensors='pt')
model = AutoModelForCausalLM.from_pretrained(model_name)
outputs = model.generate(**inputs, num_beams=5, max_new_tokens=50)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Correct this to standard English:\n slice WineBottle\nThis will give you a list of all the WineBottles you have in your inventory. If you have more than one WineBottle in your inventory, you will get a list of all the WineBottles you have in your inventory. If you']

# openai api

앗... 여러개 할 수 있었음

In [2]:
import openai
from utils import config

openai.api_key = config.OPENAI['api_key']
openai.organization = config.OPENAI['organization']

prompt = ["Correct this to standard English:\n\n%s\n"%('abs'), "Correct this to standard English:\n\n%s\n"%('do something'), "Correct this to standard English:\n\n%s\n"%('can you help me out?')]

response = openai.Completion.create(
                model = 'text-davinci-003',
                prompt = prompt,
                temperature = 0,
                logprobs=1,
                n = 1,
                max_tokens = 60,
                stop = None
            )


In [5]:
print([c.text for c in response.choices])

['\nAbs', '\nDo something.', '\nCan you help me?']


In [3]:
openai.Model.list()

<OpenAIObject list at 0x7f41adecbcb0> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sa

# Memo

In [14]:
import json
import pprint

threas = 1/100
pp = pprint.PrettyPrinter()
confusion = json.load(open('confusion_matrix_valid_seen.json', 'r'))
for key, graph in confusion.items():
    for _from in graph:
        for _to in graph[_from]:
            freq = graph[_from][_to]
            if freq > threas*820:
                print(f"{key}: {_from} -> {_to}")

total: Knife -> Apple
total: Knife -> Tomato
total: Knife -> Lettuce
total: SideTable -> DiningTable
total: ButterKnife -> Tomato
total: ButterKnife -> Apple
total: Spoon -> Ladle
total: Cup -> Mug
object_target: Knife -> Apple
object_target: Knife -> Tomato
object_target: Knife -> Lettuce
object_target: ButterKnife -> Tomato
object_target: ButterKnife -> Apple
object_target: Spoon -> Ladle
mrecep_target: Cup -> Mug
parent_target: SideTable -> DiningTable
